# Exploration into building a search function

Use a download of wikipedia pages as the data source

Build a function that searches for a target string, a simple grep analogue

Output a CSV file that contains the path, line, character index, and context of the searched string

This will be done using a multithreaded approach, my first independant attempt at multithreading.

The 4 thread approach completed the task 2.5x faster than single thread on my machine.



In [41]:
import os
import math
import functools
from multiprocessing import Pool
import pandas as pd
import time
#random used to random sample some fo the data
import random
random.seed(10)

In [42]:
folder_name = 'wiki'
file_names = os.listdir('wiki')
print(len(file_names), 'files')
with open(os.path.join(folder_name, file_names[0])) as f:
    lines = [lines for lines in f.readlines()]
file_names[:5]

1805 files


['2023_Challenger_Banque_Nationale_de_Saguenay_%E2%80%93_Singles.html',
 'Constitutio_domus_regis.html',
 'Jonathan_Biss.html',
 'Setanta_Golf.html',
 'Ivanka_Ninova.html']

Create a function that counts the number of lines
---
First step

Build a simple multithreaded function that accesses the number of lines per file

Intent: simply get the multithreaded functionality working.

In [43]:
def make_chunks(data, num_chunks):
    chunk_size = math.ceil(len(data) / num_chunks)
    return [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

def map_reduce(data, num_processes, mapper, reducer):
    chunks = make_chunks(data, num_processes)
    with Pool(num_processes) as pool:
        chunk_results = pool.map(mapper, chunks)
    return functools.reduce(reducer, chunk_results)

In [44]:
def map_num_lines(file_names_chunk):
    num_lines = 0
    for file_name in file_names_chunk:
        with open(os.path.join(folder_name, file_name)) as file:
            num_lines += len(file.readlines())
    return num_lines

def reducer_num_lines(num_lines1, num_lines2):
    return num_lines1 + num_lines2

Total lines : 1685770
---
1805 files used. The scraping script only scrapes 1000 pages. It was mistakenly ran twice and exited partway through the second one.

In [45]:
total_lines = map_reduce(file_names, 4, map_num_lines, reducer_num_lines)
total_lines

1685770

Modify the Function to give a search result
---

Returns a dict that contains {'filename':[list of rows with the searched term]}

Intent: increase funtion complexity to include the search term

In [107]:
target = input("Search for string: ")

Search for string:  blue


In [108]:
def map_search_lines(file_names_chunk):
    search_result = {}
    for file_name in file_names_chunk:
        with open(os.path.join(folder_name, file_name)) as file:
            indices = [index for index, line in enumerate(file.readlines()) if target in line]
        search_result[file_name] = indices    
    return search_result

def reducer_search_lines(search_result1, search_result2):
    merged = {}
    merged.update(search_result1)
    merged.update(search_result2)
    return merged

In [109]:
total_target_string = map_reduce(file_names, 4, map_search_lines, reducer_search_lines)

In [129]:
sample = [random.choice(list(total_target_string.keys())) for _ in range(10)]

In [135]:
# random sample some of this dict
# each dict value contains a list of row indicies that contain the target word in that file.
# random sample 10 items of the dict, 1805 items

[f'{key} : {total_target_string[key]}' for key in sample]

['Ed_McClanahan.html : []',
 'Kasbah_Boulaouane.html : []',
 'Freddy_Bravo.html : []',
 'Dennis_Dreith.html : []',
 'Morgan_%2B4%2B.html : []',
 'Ben_Verbong.html : []',
 'Innocent_Man_(Mark_Morrison_album).html : [7, 543, 549, 558, 623, 662]',
 'Corfu.html : [1831, 2121, 2122]',
 'Ever_Since_Darwin.html : []',
 'Nadine_Ellis.html : []']

Update search to be case insensative
---

Improve search function

Intent: Find additional hits were found when the search was done in a case insensative manner

In [132]:
def map_search_case_insensative(file_names_chunk):
    search_result = {}
    for file_name in file_names_chunk:
        with open(os.path.join(folder_name, file_name)) as file:
            indices = [index for index, line in enumerate(file.readlines()) if target.lower() in line.lower()]
        search_result[file_name] = indices    
    return search_result

def reducer_search_case_insensative(search_result1, search_result2):
    merged = {}
    merged.update(search_result1)
    merged.update(search_result2)
    return merged

In [136]:
total_case_insensative = map_reduce(file_names, 4, map_search_case_insensative, reducer_search_case_insensative)
# random sample some of this dict
# each dict value contains a list of row indicies that contain the target word in that file. case insensative
# random sample 10 items of the dict, 1805 items
[f'{key} : {total_case_insensative[key]}' for key in sample]


['Ed_McClanahan.html : []',
 'Kasbah_Boulaouane.html : []',
 'Freddy_Bravo.html : [537]',
 'Dennis_Dreith.html : []',
 'Morgan_%2B4%2B.html : []',
 'Ben_Verbong.html : []',
 'Innocent_Man_(Mark_Morrison_album).html : [7, 543, 549, 558, 623, 662]',
 'Corfu.html : [1827, 1831, 1929, 2121, 2122, 2688]',
 'Ever_Since_Darwin.html : []',
 'Nadine_Ellis.html : [913]']

In [137]:
#Additional hits per line that had multiple hits
#A results of 1 means there was 1 additional (2 total) hits in this line

case_inse_additional = {}
for key in total_case_insensative:
    additional = [i for i in total_case_insensative[key] if i not in total_target_string[key]]
    case_inse_additional[key] = len(additional)
    
    if case_inse_additional[key] != 0:
        print(key, ':', case_inse_additional[key])

Sundargarh_Lok_Sabha_constituency.html : 1
Washington%27s_Headquarters_State_Historic_Site.html : 1
Billy_Rich.html : 4
2020_Conference_USA_women%27s_basketball_tournament.html : 1
Guo_Longchen.html : 1
Tekkonkinkreet.html : 2
Shadow_Man_(song).html : 2
2nd_Minnesota_Cavalry_Regiment.html : 1
UTSA_Roadrunners_football.html : 1
Allan_W._Eckert.html : 3
Sherpix.html : 3
Indie_Game_Developer_Network.html : 3
Deaths_in_January_2008.html : 1
2022%E2%80%9323_Czech_First_League.html : 1
Adriaan_Carelse.html : 3
Muhammad_Salaam.html : 2
Renato_Marazzi.html : 1
Vadim_Tolstopyatov.html : 1
James_McKnight_(American_football).html : 1
Mus%C5%8D_Jikiden_Eishin-ry%C5%AB.html : 2
1900_Lafayette_football_team.html : 1
1897%E2%80%9398_Irish_League.html : 1
2023%E2%80%9324_Tweede_Divisie.html : 1
Kar%C3%A8n_Shainyan.html : 1
The_Venetian_Las_Vegas.html : 6
Hampton_Down_Stone_Circle.html : 1
When_Christmas_Comes.html : 1
Thonny.html : 1
1972%E2%80%9373_Georgetown_Hoyas_men%27s_basketball_team.html : 2
Wh

Improve search result 
---
Return Given additional hits per line, return a list of tuples that provide (line index, string index) per hit per file

This will allow us to use a loop to grab context from the files later

In [141]:
#Added helper function to clean up the list comprehension used for bulding the search result tuples

def get_index_tuples(line, line_num):
    index_tuples = [(line_num, index) for index in range(len(line)) if line.startswith(target.lower(), index)]
    return index_tuples

def map_search_line_tuples(file_names_chunk):
    search_result = {}
    for file_name in file_names_chunk:
        with open(os.path.join(folder_name, file_name)) as file:
            index_tuples = [get_index_tuples(line.lower(), index) for index, line in enumerate(file.readlines()) if target.lower() in line.lower()]
        search_result[file_name] = index_tuples    
    return search_result

def reducer_search_line_tuples(search_result1, search_result2):
    merged = {}
    merged.update(search_result1)
    merged.update(search_result2)
    return merged

In [142]:
total_tuples = map_reduce(file_names, 4, map_search_line_tuples, reducer_search_line_tuples)

[f'{key} : {total_tuples[key]}' for key in sample]

['Ed_McClanahan.html : []',
 'Kasbah_Boulaouane.html : []',
 'Freddy_Bravo.html : [[(537, 495), (537, 516), (537, 661), (537, 682), (537, 760), (537, 781)]]',
 'Dennis_Dreith.html : []',
 'Morgan_%2B4%2B.html : []',
 'Ben_Verbong.html : []',
 'Innocent_Man_(Mark_Morrison_album).html : [[(7, 41)], [(543, 3948), (543, 4617)], [(549, 100)], [(558, 243)], [(623, 1227), (623, 1252)], [(662, 441), (662, 493), (662, 529)]]',
 'Corfu.html : [[(1827, 364), (1827, 400)], [(1831, 734), (1831, 781), (1831, 801)], [(1929, 861), (1929, 921), (1929, 954)], [(2121, 139), (2121, 180)], [(2122, 109), (2122, 128)], [(2688, 443), (2688, 904)]]',
 'Ever_Since_Darwin.html : []',
 'Nadine_Ellis.html : [[(913, 24)]]']

Construct into a functional datatype
--
Data structure:|File Path |Line Index | String Index| String Context|

Build a non-mutlithreaded implementation first, then construct into a multithreaded function.

Compare time taken to do the context calls and build the dataframe

In [143]:
#Build non-multithreaded DataFrame constructor
start = time.time()
rows = []
for file_name in total_tuples:
    with open(os.path.join(folder_name, file_name)) as file:
        lines = file.readlines()
        for line_list in total_tuples[file_name]:
            for tup in line_list:
                context = lines[tup[0]][tup[1]-20 : tup[1]+10+len(target)]
                row = [''.join([folder_name, '/', file_name]), tup[0], tup[1], context]
                rows.append(row)
                
df = pd.DataFrame(rows, columns=['File', 'Line', 'Index', 'Context'])
print(time.time()-start, 's')
df

0.19721603393554688 s


,File,Line,Index,Context
0,wiki/Sundargarh_Lok_Sabha_constituency.html,726,41,"; background-color: Blue;"" data-so"
1,wiki/Washington%27s_Headquarters_State_Histori...,975,19,
2,wiki/Washington%27s_Headquarters_State_Histori...,975,44,"kin_(horse)"" title=""Blueskin (hors"
3,wiki/Washington%27s_Headquarters_State_Histori...,975,62,"=""Blueskin (horse)"">Blueskin</a></"
4,wiki/2024_Tour_of_the_Alps.html,1973,28,"e=""background:dodgerblue;""><a href"
...,...,...,...,...
3223,wiki/Gary_Waddock.html,1160,1819,"e,inset -1px -1px 0 blue; width: 8"
3224,wiki/Gary_Waddock.html,1261,522,"e=""background:mediumblue; color:wh"
3225,wiki/Gary_Waddock.html,1261,1010,"="";background:mediumblue; color:wh"
3226,wiki/Gary_Waddock.html,1261,1376,"="";background:mediumblue; color:wh"


In [145]:
#Build multithreaded DataFrame constructor
#Could not pass the dictionary or keys directly as Python dicionary keys are not ordered, and can not be subscripted
def map_to_df(file_names_chunk):
    rows = []
    for file_name in file_names_chunk:
        with open(os.path.join(folder_name, file_name)) as file:
            lines = file.readlines()
            for line_list in total_tuples[file_name]:
                for tup in line_list:
                    context = lines[tup[0]][tup[1]-20 : tup[1]+10+len(target)]
                    row = [''.join([folder_name, '/', file_name]), tup[0], tup[1], context]
                    rows.append(row)
    return rows

def reducer_to_df(rows1, rows2):
    merged = []
    merged.extend(rows1)
    merged.extend(rows2)
    return merged

start = time.time()

to_df = map_reduce(file_names, 4, map_to_df, reducer_to_df)
df = pd.DataFrame(to_df, columns=['File', 'Line', 'Index', 'Context'])

print(time.time()-start, 's')
df

0.07882881164550781 s


,File,Line,Index,Context
0,wiki/Sundargarh_Lok_Sabha_constituency.html,726,41,"; background-color: Blue;"" data-so"
1,wiki/Washington%27s_Headquarters_State_Histori...,975,19,
2,wiki/Washington%27s_Headquarters_State_Histori...,975,44,"kin_(horse)"" title=""Blueskin (hors"
3,wiki/Washington%27s_Headquarters_State_Histori...,975,62,"=""Blueskin (horse)"">Blueskin</a></"
4,wiki/2024_Tour_of_the_Alps.html,1973,28,"e=""background:dodgerblue;""><a href"
...,...,...,...,...
3223,wiki/Gary_Waddock.html,1160,1819,"e,inset -1px -1px 0 blue; width: 8"
3224,wiki/Gary_Waddock.html,1261,522,"e=""background:mediumblue; color:wh"
3225,wiki/Gary_Waddock.html,1261,1010,"="";background:mediumblue; color:wh"
3226,wiki/Gary_Waddock.html,1261,1376,"="";background:mediumblue; color:wh"


In [148]:
df.to_csv()
#supress to_csv output
None